# Swallowつかってみよう

公式ページ
https://tokyotech-llm.github.io/

Huggingface(70bのやつ)
https://huggingface.co/tokyotech-llm/Swallow-70b-instruct-hf

ブログ記事（詳細情報など）
https://zenn.dev/tokyotech_lm/articles/d6cb3a8fdfc907

使うときはRuntime をA100にしないとうまくゆかないかも。
V100だとメモリのエラーがでてしまった。


## Huggingfaceに書いてあるやつそのままやってみたよ

In [ ]:
!pip install torch transformers sentencepiece accelerate protobuf langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.1 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 他にも13bと70bのパラメータで作ったモデルがある。tokyotech-llm/Swallow-13b-instruct-hf とかSwallow-70b-instruct-hfとか
# 7bだとモデルのロードがすぐ終わるけど…
model_name = "tokyotech-llm/Swallow-7b-instruct-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True, device_map="auto")


PROMPT_DICT = {
    "prompt_input": (
        "以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 入力:\n{input}\n\n### 応答:"

    ),
    "prompt_no_input": (
        "以下に、あるタスクを説明する指示があります。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 応答:"
    ),
}

def create_prompt(instruction, input=None):
    """
    Generates a prompt based on the given instruction and an optional input.
    If input is provided, it uses the 'prompt_input' template from PROMPT_DICT.
    If no input is provided, it uses the 'prompt_no_input' template.

    Args:
        instruction (str): The instruction describing the task.
        input (str, optional): Additional input providing context for the task. Default is None.

    Returns:
        str: The generated prompt.
    """
    if input:
        # Use the 'prompt_input' template when additional input is provided
        return PROMPT_DICT["prompt_input"].format(instruction=instruction, input=input)
    else:
        # Use the 'prompt_no_input' template when no additional input is provided
        return PROMPT_DICT["prompt_no_input"].format(instruction=instruction)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
instruction_example = "以下のトピックに関する詳細な情報を提供してください。"
input_example = "日本の著作権保護期間は何年？"
prompt = create_prompt(instruction_example, input_example)

input_ids = tokenizer.encode(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
)

tokens = model.generate(
    input_ids.to(device=model.device),
    max_new_tokens=500,
    temperature=0.99,
    top_p=0.95,
    do_sample=True,
)

out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)

以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。リクエストを適切に完了するための回答を記述してください。

### 指示:
以下のトピックに関する詳細な情報を提供してください。

### 入力:
日本の著作権保護期間は何年？

### 応答:日本の著作権保護期間は70年です。


## __なんとか応用できないものか__



# 注意
GPUの設定をA100にしないとRAM不足でエラーがでるかも。V100のRAM上限20GMだととまる…



[JaLCのAPI](https://api.japanlinkcenter.org/api-docs/index.html)でDOIのリストをゲット

[J-StageのAPI](https://www.jstage.jst.go.jp/static/pages/WebAPI/-char/ja)でもできる。
ただし論文要旨とか抄録とかの情報はどちらからも取れない…


ので、JaLCのAPIから例えば史学雑誌の新しい記事のDOIをゲットして、
URLからURLを取得してウェブページをスクレイピング



In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


def getAbstract(doi):
    res = requests.get(doi)
    sp = BeautifulSoup(res.content)

    abst_tag = sp.find("meta", attrs={"name": "abstract"})
    if abst_tag is None:
        title_string = None
        abst_string = None
    else:
        abst_string = abst_tag["content"]
        title_string = sp.find("title").text

    return title_string, abst_string


jalc_link = "https://api.japanlinkcenter.org/doilist/10.24471?rows=50&page=1&sort=updated_date&order=desc"
re = requests.get(jalc_link)
json_data = re.json()

items = json_data["data"]["items"]
dois = []
for i in items:

    url = i["dois"]["url"]
    dois.append(url)

tit_list = []
abst_list = []
link_list = []
for j in dois:

    if "cover" in j:
        continue
    else:
        tit, abst = getAbstract(j)
        tit_list.append(tit)
        abst_list.append(abst)
        link_list.append(j)
        time.sleep(3)

df = pd.DataFrame(list(zip(tit_list, abst_list, link_list)), columns=["title", "abstract", "link"])
df_without_none = df.dropna()
#print(df_without_none)
df_without_none.to_csv("abstract_table.csv", encoding="utf-8", index=False)


In [7]:
# langchain_openai のライブラリをつかったときにエラーがでたのでそのためにこれ
# https://community.openai.com/t/error-while-importing-openai-from-open-import-openai/578166/27
%%writefile /usr/local/lib/python3.10/dist-packages/openai/_utils/_streams.py
from typing import Any
from typing_extensions import AsyncIterator
from typing import Iterator # import Iterator from the correct library

def consume_sync_iterator(iterator: Iterator[Any]) -> None:
    for _ in iterator:
        ...

async def consume_async_iterator(iterator: AsyncIterator[Any]) -> None:
    async for _ in iterator:
        ...

Overwriting /usr/local/lib/python3.10/dist-packages/openai/_utils/_streams.py


In [3]:
!pip install -U langchain-community faiss-cpu langchain-openai tiktoken cohere langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 100.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import os
os.environ["OPENAI_API_KEY"] = "[OpenAIのAPIキー]"

In [8]:
import pandas as pd

from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

article_df = pd.read_csv("abstract_table.csv")
docs = []
for index, row in article_df.iterrows():

    title = row["title"]
    abst = row["abstract"]
    link = row["link"]

    metadata = {"article_title": title, "link": link}
    doc_obj = Document(page_content=abst, metadata=metadata)
    docs.append(doc_obj)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding=embeddings)
vectorstore.save_local("faiss_index")


## FAISSのベクトルストアをアップロードして使いたいとき


In [1]:
# faissのベクトルストアをアップロードして使いたい時。faiss_indexというフォルダを作ってそこにindex.faissとindex.pklファイルをアップロードする
!mkdir faiss_index

faiss_indexのフォルダ下にデータをアップロードしてね。サンプルは[ここ](https://github.com/NbtKmy/swallow_trial)のリポジトリの"materials">"faiss_index"にあります。

In [ ]:
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.load_local("faiss_index", embeddings)

In [9]:
results_with_scores = vectorstore.similarity_search_with_score("中国の歴史")
for doc, score in results_with_scores:
    #print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")
    metadata = doc.metadata
    print(metadata["link"])

https://doi.org/10.24471/shigaku.131.10_1
https://doi.org/10.24471/shigaku.131.10_24
https://doi.org/10.24471/shigaku.131.6_35
https://doi.org/10.24471/shigaku.131.11_66


## Langchain + Swallow

In [10]:
!pip install torch transformers sentencepiece accelerate protobuf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.0 MB/s eta 0:00:00


In [11]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "tokyotech-llm/Swallow-7b-instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True, device_map="auto")
pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128
)
llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.77G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/203 [00:00<?, ?B/s]

In [16]:
from langchain import PromptTemplate, LLMChain

prompt_template = """
        ### 指示:\n{instruction}\n\n### 質問:\n{input}\n\n### 応答:
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["instruction", "input"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

input = "中国の歴史に関する論文を一つ紹介して"

source = ""
results_with_scores = vectorstore.similarity_search_with_score(input)
for doc, score in results_with_scores:
    abst = "概要: " + doc.page_content
    metadata = doc.metadata
    article = "記事タイトル: " + metadata["article_title"]
    link = "リンク: " + metadata["link"]
    row = abst + article + link
    source += "\n" + row

instruction = "質問に対して以下の文献を参考にしつつ答えを作成してください" + source

result = llm_chain.invoke({"instruction": instruction, "input": input})

print(result["text"])

中国の歴史に関する論文はたくさんありますが、私が選んだのは、中国の啓という文書様式についての論文です。この論文は、中国の啓という文書様式について、史料中の「啓」という語句の分析を手懸りに、中国における啓との比較を通じて検討したものである。
